<h1 align="center">
    Topic Modeling of Pet Service Reviews <br> Using BERTimbau and HDBSCAN
</h1>

*******************************************************************************************************************************

<h2>1. Introduction</h2>

This project focuses on **topic modeling** of customer reviews written in **Brazilian Portuguese**, specifically from pet-related businesses located in **Santo André/SP, Brazil**. Online reviews often contain informal, context-dependent language, which presents a challenge for extracting meaningful themes — particularly in languages other than English.

To tackle this challenge, we designed a modern NLP pipeline centered around **BERTimbau**, a transformer-based language model pre-trained on Brazilian Portuguese. BERTimbau was used to generate **dense semantic embeddings** that capture the underlying meaning and nuance of each review, going beyond simple keyword matching.

To identify coherent thematic structures within the review data, we applied **HDBSCAN**, a clustering algorithm well-suited for high-dimensional, text-based embeddings. HDBSCAN was chosen because it **automatically detects the number of clusters** based on the semantic density of the data, adapting to areas with varying complexity. This made it a flexible and robust choice for modeling the diversity of customer opinions.

After clustering, **BERTimbau** was used again — this time to extract the most relevant keywords from each cluster. Its strong pretraining on Portuguese helped highlight key terms that accurately reflect the core content of each group, especially when dealing with subtle or informal language.

To make the topics easier to interpret, we then applied **KeyBERT**, a keyword extraction library that selects representative terms by comparing them with the original embeddings. This step enabled us to generate concise, human-readable labels that summarize each cluster's central idea.

For clearer insights, the reviews were divided into *positive* and *negative* categories based on their star ratings. Topic modeling was performed separately on each group, allowing us to compare recurring themes in positive versus negative feedback, and observe which aspects customers tend to praise or criticize most.

This pipeline — combining **contextual embeddings**, **adaptive clustering**, and **keyword summarization** — allowed us to highlight common concerns, expectations, and experiences shared by customers in their own words, providing a detailed view of feedback within the pet care sector.

<h2>2. Initialization</h2>

In [ ]:
# Library Imports
import os
import csv
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import defaultdict

import torch
from transformers import BertTokenizer, BertModel

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity

from umap import UMAP
from hdbscan import HDBSCAN

import langid

import optuna
from optuna.visualization import plot_optimization_history
from optuna.samplers import TPESampler

from keybert import KeyBERT

<h2>3. Load the Dataset</h2>

In [ ]:
PATH = os.path.abspath(os.path.join("..", "results", "predictions", "reviews_with_predictions.csv"))

In [ ]:
reviews_df = pd.read_csv(PATH, sep=";", header=0, encoding="utf-8")

In [ ]:
reviews_df.shape

In [ ]:
reviews_df.columns

In [ ]:
reviews_df.head(3)

<h2>4. Data Preprocessing</h2>

<h3>4.1 Filtering Out Short Reviews</h3>

In [ ]:
def filter_reviews_by_min_words(df, text_column="Text", min_words=4):
    """
    Filters a DataFrame to keep only reviews with at least a specified number of words.

    Args:
        df (pd.DataFrame): The DataFrame containing the reviews.
        text_column (str): The name of the column containing the review texts.
        min_words (int): Minimum number of words required for a review to be kept. Default is 4.

    Returns:
        pd.DataFrame: A filtered DataFrame with only reviews that meet the minimum word count.
    """
    return df[df[text_column].apply(lambda x: isinstance(x, str) and len(x.strip().split()) >= min_words)]

In [ ]:
# Filter the DataFrame to keep only reviews with at least 4 words
reviews_df = filter_reviews_by_min_words(reviews_df, text_column="Text", min_words=4)

<h3>4.2 Excluding Reviews in English</h3>

In [ ]:
def is_english(text):
    """
    Detects whether a given text is in English using langid.

    Args:
        text (str): The text to analyze.

    Returns:
        bool: True if the text is detected as English, False otherwise.
    """
    if not isinstance(text, str) or text.strip() == "":
        return False
    lang, _ = langid.classify(text)
    return lang == "en"

In [ ]:
# Remove rows where the review text is in English
reviews_df = reviews_df[~reviews_df["Text"].apply(is_english)].copy()

<h3>4.3 Categorizing Ratings into Sentiment Labels</h3>

In [ ]:
# Categorize ratings into "Negative", "Positive", and "Neutral" sentiments
reviews_df["Sentiment"] = reviews_df["Predicted Rating"].map(
    lambda x: "Negative" if x in [1, 2] 
    else "Positive" if x in [4, 5] 
    else "Neutral"
)

In [ ]:
# Separate texts
positive_reviews = reviews_df[reviews_df["Sentiment"] == "Positive"]["Text"].tolist()
negative_reviews = reviews_df[reviews_df["Sentiment"] == "Negative"]["Text"].tolist()
neutral_reviews = reviews_df[reviews_df["Sentiment"] == "Neutral"]["Text"].tolist()

<h3>4.4 Distribution of Sentiments</h3>

In [ ]:
def save_plot(fig, filename="sentiment_distribution.png"):
    """
    Saves the given figure to a file.

    Args:
        fig (matplotlib.figure.Figure): The figure to save.
        filename (str): The name of the file to save the plot to. Default is 'sentiment_distribution.png'.
    """
    # Save the figure to the specified file
    fig.savefig(filename, bbox_inches='tight')  # Save with tight bounding box to avoid clipping

    # Close the figure (so it doesn't display in the notebook)
    plt.close(fig)

In [ ]:
def bar_plot(x, y):
    """
    Creates a bar plot to visualize the distribution of sentiments.

    Args:
        x (list or array-like): The categories for the x-axis (e.g., sentiment labels).
        y (list or array-like): The corresponding counts or frequencies for each category.

    Returns:
        None
    """
    # Create a figure and axis with a predefined size
    fig, ax = plt.subplots(figsize=(5, 4))
    
    # Define the width of the bars
    bar_width = 0.85 

    # Set the title and label for the x-axis
    ax.set_title("Distribution of Sentiments", fontfamily='Arial Rounded MT Bold', fontsize=16, pad=15)
    ax.set_xlabel("Sentiment", fontsize=12)

    # Add a grid for better readability (dashed lines, gray color, semi-transparent)
    ax.grid(visible=True, linestyle='--', color='gray', alpha=0.7)

    # Convert the x values to a NumPy array to ensure compatibility with plotting
    x = np.array(x)  

    # Create the bar plot with a custom color and border
    bars = ax.bar(x, y, width=bar_width, color="#3498db", edgecolor='#e6e6e6')

    # Annotate each bar with its height (value)
    for bar in bars:
        yval = bar.get_height()  # Retrieve the height of the bar
        ax.text(
            bar.get_x() + bar.get_width() / 2, yval, round(yval, 2), 
            ha='center', va='bottom', color='black', fontsize=10
        )  

    # Set the x-axis tick labels based on the provided x values
    ax.set_xticks(x)

    # Display the plot
    plt.show()

    # Save the plot
    save_plot(fig, os.path.join("..", "results", "figures", "topic_modeling", "sentiment_distribution.png"))

In [ ]:
# Generate a bar plot showing the distribution of sentiments
bar_plot(
    reviews_df['Sentiment'].value_counts().sort_index().index, 
    reviews_df['Sentiment'].value_counts().sort_index().values
)

<h2>5. Generating Embeddings with BERTimbau</h2>

<h3>5.1 Load BERTimbau Model and Tokenizer</h3>

In [ ]:
# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define the pre-trained model name
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"

# Load the tokenizer for the selected BERT model
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Load the BERT model and move it to the appropriate device (CPU/GPU)
model = BertModel.from_pretrained(MODEL_NAME).to(device)

<h3>5.2 Generating Text Embeddings</h3>

In [ ]:
def compute_embeddings(reviews, tokenizer, model, device, batch_size=32):
    """
    Generates BERT embeddings for a list of reviews using the [CLS] token.

    Args:
        reviews (list): List of review texts.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer from the BERTimbau model.
        model (transformers.PreTrainedModel): BERTimbau model for generating embeddings.
        device (str or torch.device): Device to run the model on (e.g., "cpu" or "cuda").
        batch_size (int, optional): Number of reviews processed per batch. Default is 32.

    Returns:
        np.ndarray: Matrix containing the embeddings for all reviews.
    """
    model.to(device)
    model.eval()
    embeddings = []

    # Process reviews in batches
    for i in tqdm(range(0, len(reviews), batch_size), desc="Generating embeddings"):
        batch = reviews[i : i + batch_size]

        # Tokenize the batch with padding and truncation
        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():  # Disable gradient computation for efficiency
            outputs = model(**inputs)

        # Extract the [CLS] token embeddings (first token of each sequence)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)

    # Stack all batch embeddings into a single matrix
    return np.vstack(embeddings)

In [ ]:
# Generate embeddings for positive and negative keywords
positive_embeddings = compute_embeddings(positive_reviews, tokenizer, model, device)
negative_embeddings = compute_embeddings(negative_reviews, tokenizer, model, device)

<h2>6. Clustering Reviews with HDBSCAN</h2>

<h3>6.1 Standard Scaling of Embeddings</h3>

In [ ]:
def scale_embeddings(embeddings):
    """
    Applies standard scaling (zero mean, unit variance) to the embeddings.

    Args:
        embeddings (np.ndarray): Input embeddings.

    Returns:
        np.ndarray: Scaled embeddings.
    """
    # Initialize a StandardScaler, which scales features to have zero mean and unit variance
    scaler = StandardScaler()

    # Fit the scaler to the embeddings and transform them
    return scaler.fit_transform(embeddings)

In [ ]:
# Apply Standard Scaler to the BERT embeddings for positive and negative reviews
scaled_positive_embeddings = scale_embeddings(positive_embeddings)
scaled_negative_embeddings = scale_embeddings(negative_embeddings)

<h3>6.2 Hyperparameter Optimization of UMAP and HDBSCAN Using Optuna</h3>

In [ ]:
def objective(trial, X, param_ranges):
    """
    Objective function for Optuna hyperparameter optimization using UMAP and HDBSCAN.

    Args:
        trial (optuna.trial.Trial): An Optuna trial object to suggest hyperparameters.
        X (array-like): Input data (embeddings) for dimensionality reduction and clustering.
        param_ranges (dict): Dictionary containing ranges for hyperparameter tuning. Expected keys:
            - "n_neighbors" (tuple): Min and max values for UMAP's n_neighbors.
            - "min_dist" (tuple): Min and max values for UMAP's min_dist.
            - "n_components" (list): List of UMAP output dimensions to choose from.
            - "min_cluster_size" (tuple): Min and max values for HDBSCAN's min_cluster_size.
            - "min_samples" (tuple): Min and max values for HDBSCAN's min_samples.
            - "n_clusters" (tuple): Acceptable range for the number of clusters.

    Returns:
        float: The silhouette score of the clustering result (higher is better).
               Returns -1.0 if the configuration is penalized due to undesirable results.
    """
    # Define hyperparameters using the provided parameter ranges
    n_neighbors = trial.suggest_int("n_neighbors", *param_ranges["n_neighbors"])
    min_dist = trial.suggest_float("min_dist", *param_ranges["min_dist"])
    n_components = trial.suggest_categorical("n_components", param_ranges["n_components"])
    min_cluster_size = trial.suggest_int("min_cluster_size", *param_ranges["min_cluster_size"])
    min_samples = trial.suggest_int("min_samples", *param_ranges["min_samples"])

    # Apply UMAP for dimensionality reduction (using cosine distance)
    umap_model = UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric='cosine',
        n_components=n_components,
        random_state=42
    )
    X_umap = umap_model.fit_transform(X)

    # Apply HDBSCAN clustering (using Euclidean distance)
    clusterer = HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric='euclidean',
        cluster_selection_method='eom'
    )
    labels = clusterer.fit_predict(X_umap)

    # Compute the number of clusters and the outlier ratio
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_outliers = np.sum(labels == -1)
    outlier_ratio = n_outliers / len(labels)

    # Retrieve cluster count limits from param_ranges
    min_clusters, max_clusters = param_ranges["n_clusters"]

    # Penalize configurations with too few/many clusters or too many outliers
    if n_clusters < min_clusters or n_clusters > max_clusters or outlier_ratio > 0.3:
        return -1.0

    # Compute silhouette score for clustering quality assessment
    score = silhouette_score(X_umap, labels)

    # Penalize negative silhouette scores
    if score < 0:
        return -1.0

    return score

In [ ]:
def plot_optuna_history(study):
    """
    Plots the optimization history of an Optuna study.

    Args:
        study (optuna.study.Study): The Optuna study to visualize.
    """
    # Generate the optimization history plot using Optuna's built-in function
    fig = plot_optimization_history(study)

    # Display the interactive plot
    fig.show()

In [ ]:
def optimize_umap_hdbscan_with_optuna(X, param_ranges, study_name, n_trials=100, seed=42):
    """
    Optimizes UMAP and HDBSCAN hyperparameters using Optuna.

    Args:
        X (array-like): Input data (embeddings) to be used for dimensionality reduction and clustering.
        param_ranges (dict): Dictionary containing the hyperparameter ranges for tuning. Must include:
            - "n_neighbors": Tuple[int, int] - Range for UMAP's number of neighbors.
            - "min_dist": Tuple[float, float] - Range for UMAP's minimum distance.
            - "n_components": List[int] - Options for UMAP's output dimensions.
            - "min_cluster_size": Tuple[int, int] - Range for HDBSCAN's minimum cluster size.
            - "min_samples": Tuple[int, int] - Range for HDBSCAN's minimum samples.
            - "n_clusters": Tuple[int, int] - Acceptable range for the number of clusters.
        study_name (str): Name for the Optuna study.
        n_trials (int, optional): Number of trials to run in the optimization. Defaults to 100.
        seed (int, optional): Random seed for reproducibility. Defaults to 42.

    Returns:
        optuna.study.Study: The Optuna study object containing optimization results.
    """
    # Define sampler with fixed seed for reproducibility
    sampler = TPESampler(seed=seed)

    # Create an Optuna study (or load existing one) to maximize the silhouette score
    study = optuna.create_study(
        study_name=study_name,
        direction="maximize",
        sampler=sampler,
        storage="sqlite:///../logs/optuna/optuna_topic_modeling.db",
        load_if_exists=True
    )

    # Run the optimization trials
    study.optimize(lambda trial: objective(trial, X, param_ranges), n_trials=n_trials)

    # Display best hyperparameters and score
    print("Best hyperparameters found:")
    print(study.best_params)
    print("Best silhouette score:", study.best_value)

    return study

In [ ]:
# Define hyperparameter ranges for clustering on positive sentiment reviews
positive_param_ranges = {
    "n_neighbors": (5, 10),
    "min_dist": (0.0, 0.01),
    "n_components": [3, 5, 10],
    "min_cluster_size": (50, 150),
    "min_samples": (10, 40),
    "n_clusters": (10, 30)
}

In [ ]:
# Run Optuna optimization for clustering on positive sentiment embeddings
positive_optimization_results = optimize_umap_hdbscan_with_optuna(scaled_positive_embeddings, 
                                                                  positive_param_ranges,
                                                                  study_name="topic_modeling_positive")

In [ ]:
# Plot the optimization history of the positive embeddings results
plot_optuna_history(positive_optimization_results)

In [ ]:
# Define hyperparameter ranges for clustering on negative sentiment reviews
negative_param_ranges = {
    "n_neighbors": (5, 10),
    "min_dist": (0.0, 0.01),
    "n_components": [3, 5, 10],
    "min_cluster_size": (10, 60),
    "min_samples": (5, 25),
    "n_clusters": (5, 15)
}

In [ ]:
# Run Optuna optimization for clustering on positive sentiment embeddings
negative_optimization_results = optimize_umap_hdbscan_with_optuna(scaled_negative_embeddings, 
                                                                  negative_param_ranges, 
                                                                  study_name="topic_modeling_negative")

In [ ]:
# Plot the optimization history of the negative embeddings results
plot_optuna_history(negative_optimization_results)

<h3>6.3 Dimensionality Reduction with UMAP</h3>

In [ ]:
def reduce_with_umap(embeddings, n_components=2, n_neighbors=15, min_dist=0.0, random_state=42):
    """
    Reduces high-dimensional embeddings using UMAP.

    Args:
        embeddings (np.ndarray): Input high-dimensional vectors, typically scaled.
        n_components (int): Number of dimensions to reduce the embeddings to (e.g., 2 for visualization).
        n_neighbors (int): Number of neighboring points used in the local approximation of the manifold.
        min_dist (float): Minimum distance allowed between points in the low-dimensional space. 
                          Smaller values preserve more local structure.
        random_state (int): Seed for reproducibility.

    Returns:
        np.ndarray: Embeddings reduced to the specified number of dimensions.
    """
    # Initialize UMAP with specified parameters
    umap_model = UMAP(
        n_components=n_components,
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric='cosine',
        random_state=random_state
    )

    # Fit UMAP on the embeddings and return the reduced output
    reduced = umap_model.fit_transform(embeddings)
    return reduced

In [ ]:
# Reduce the dimensionality of the positive and negative embeddings to 2D using UMAP
positive_umap_embeddings = reduce_with_umap(scaled_positive_embeddings, n_components=5, n_neighbors=5, min_dist=0.000685789716338362)
negative_umap_embeddings = reduce_with_umap(scaled_negative_embeddings, n_components=5, n_neighbors=10, min_dist=0.005694198925849235)

<h3>6.4 Evaluating HDBSCAN Performance Across min_cluster_size Values</h3>

In [ ]:
def evaluate_hdbscan(embeddings, min_cluster_range=range(5, 30), min_samples=21):
    """
    Evaluates HDBSCAN clustering across a range of minimum cluster sizes.

    For each `min_cluster_size` value in the provided range, the function computes:
    - The number of clusters identified (excluding noise points).
    - The Silhouette Score of the clustering (excluding noise).
    - The percentage of points classified as noise (label -1).

    Args:
        embeddings (array-like): Input array of vector embeddings (e.g., UMAP-reduced embeddings).
        min_cluster_range (Iterable[int], optional): A sequence of `min_cluster_size` values to test. 
            Defaults to range(5, 30).
        min_samples (int, optional): The number of samples in a neighborhood for a point to be considered
            a core point. Default is 21, consistent with the main project configuration.

    Returns:
        dict: A dictionary with the following keys:
            - 'min_cluster_size' (List[int]): Values of `min_cluster_size` evaluated.
            - 'num_clusters' (List[int]): Number of clusters found for each setting.
            - 'silhouette_scores' (List[float]): Silhouette Score (excluding noise) for each setting.
            - 'outlier_percentages' (List[float]): Percentage of points identified as noise for each setting.
    """
    # Initialize lists to store evaluation metrics
    num_clusters = []
    silhouette_scores = []
    outlier_percentages = []

    # Iterate over each min_cluster_size value in the specified range
    for min_size in min_cluster_range:
        # Initialize the HDBSCAN clusterer with the current min_cluster_size
        clusterer = HDBSCAN(min_cluster_size=min_size, 
                            min_samples=min_samples, 
                            metric='euclidean',
                            cluster_selection_method='eom')

        # Fit the model and predict cluster labels
        cluster_labels = clusterer.fit_predict(embeddings)

        # Calculate the number of clusters found (excluding noise)
        n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        num_clusters.append(n_clusters)

        # Create a mask for non-noise points (i.e., points not labeled as -1)
        mask = cluster_labels != -1
        if mask.sum() > 1:
            # Compute the Silhouette Score for non-noise points
            score = silhouette_score(embeddings[mask], cluster_labels[mask])
        else:
            # Assign NaN if there are too few non-noise points to compute the score
            score = np.nan
        silhouette_scores.append(score)

        # Calculate the percentage of points classified as noise
        outlier_pct = np.sum(cluster_labels == -1) / len(cluster_labels) * 100
        outlier_percentages.append(outlier_pct)

    # Compile the results into a dictionary
    results = {
        'min_cluster_size': list(min_cluster_range),
        'num_clusters': num_clusters,
        'silhouette_scores': silhouette_scores,
        'outlier_percentages': outlier_percentages
    }

    return results

In [ ]:
def plot_hdbscan_results(results, category):
    """
    Plots HDBSCAN evaluation metrics across different min_cluster_size values.

    This function generates a figure with three subplots:
    1. Number of clusters found vs. min_cluster_size.
    2. Silhouette score vs. min_cluster_size.
    3. Outlier percentage vs. min_cluster_size.

    The figure is saved to a path depending on the label provided.

    Args:
        results (dict): A dictionary containing the following keys:
            - 'min_cluster_size' (List[int]): Values of min_cluster_size tested.
            - 'num_clusters' (List[int]): Number of clusters found for each setting.
            - 'silhouette_scores' (List[float]): Silhouette score for each configuration.
            - 'outlier_percentages' (List[float]): Percentage of data points classified as outliers.
        category (str): Category label (e.g., 'positive' or 'negative') to customize output path.

    Returns:
        None: Displays and saves the matplotlib figure.
    """
    # Extract the list of min_cluster_size values from the results dictionary
    min_cluster_sizes = results['min_cluster_size']

    # Create a figure with 3 subplots arranged horizontally
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    # Set a global title for the figure
    fig.suptitle(f"HDBSCAN Clustering Performance for {category.capitalize()} Reviews",
                 fontfamily='Arial Rounded MT Bold', fontsize=18)

    # ---- Plot 1: Number of Clusters vs. min_cluster_size ----
    axs[0].plot(min_cluster_sizes, results['num_clusters'],
                marker='o', linestyle='-', color='#0d47a1')
    axs[0].set_title("Number of Clusters",
                     fontfamily='Arial Rounded MT Bold', fontsize=14)
    axs[0].set_xlabel("min_cluster_size", fontsize=12)
    axs[0].set_ylabel("Number of Clusters", fontsize=12)
    axs[0].grid(True)

    # ---- Plot 2: Silhouette Score vs. min_cluster_size ----
    axs[1].plot(min_cluster_sizes, results['silhouette_scores'],
                marker='o', linestyle='-', color='#2E7D32')
    axs[1].set_title("Silhouette Score",
                     fontfamily='Arial Rounded MT Bold', fontsize=14)
    axs[1].set_xlabel("min_cluster_size", fontsize=12)
    axs[1].set_ylabel("Silhouette Score", fontsize=12)
    axs[1].grid(True)

    # ---- Plot 3: Outlier Percentage vs. min_cluster_size ----
    axs[2].plot(min_cluster_sizes, results['outlier_percentages'],
                marker='o', linestyle='-', color='#c62828')
    axs[2].set_title("Outlier Percentage",
                     fontfamily='Arial Rounded MT Bold', fontsize=14)
    axs[2].set_xlabel("min_cluster_size", fontsize=12)
    axs[2].set_ylabel("Outlier Percentage (%)", fontsize=12)
    axs[2].grid(True)

    # Automatically adjust layout to avoid label overlap
    plt.tight_layout()

    # Display the complete figure
    plt.show()

    # Save the figure with appropriate path based on label
    output_path = os.path.join("..", "results", "figures", "topic_modeling", f"{category}_hdbscan_performance.png")
    save_plot(fig, output_path)

In [ ]:
# Evaluate HDBSCAN clustering performance on the positive UMAP embeddings
positive_hdbscan_results = evaluate_hdbscan(positive_umap_embeddings, range(50, 90), 21)

In [ ]:
# Visualize the HDBSCAN clustering evaluation results for the positive embeddings
plot_hdbscan_results(positive_hdbscan_results, category="positive")

In [ ]:
# Evaluate HDBSCAN clustering performance on the negative UMAP embeddings
negative_hdbscan_results = evaluate_hdbscan(negative_umap_embeddings, range(5, 30), 5)

In [ ]:
# Visualize the HDBSCAN clustering evaluation results for the negative embeddings
plot_hdbscan_results(negative_hdbscan_results, category="negative")

<h3>6.5 Application of the HDBSCAN Algorithm to 2D Embeddings</h3>

In [ ]:
def cluster_with_hdbscan(embeddings_2d, min_cluster_size=5, min_samples=10):
    """
    Applies the HDBSCAN clustering algorithm to 2D embeddings and returns cluster labels along with membership probabilities.

    Args:
        embeddings_2d (array-like): Data points in 2D space to be clustered.
        min_cluster_size (int, optional): Minimum number of samples in a cluster. Defaults to 5.
        min_samples (int, optional): Controls how conservative the clustering is. Higher values lead to more noise points. Defaults to 10.

    Returns:
        Tuple:
            labels (ndarray): Cluster labels assigned to each point. Points labeled as -1 are considered noise.
            probabilities_dict (dict): Dictionary with key `'probabilities'` containing the membership strength (confidence) for each point.
    """
    # Convert input embeddings to NumPy array if not already
    embeddings_2d = np.asarray(embeddings_2d)

    # Initialize the HDBSCAN clusterer with specified hyperparameters
    clusterer = HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric='euclidean',
        cluster_selection_method='eom'
    )

    # Fit the clusterer and get predicted cluster labels
    labels = clusterer.fit_predict(embeddings_2d)

    # Retrieve the soft clustering probabilities (membership strength per point)
    probabilities = clusterer.probabilities_

    # Store probabilities in a dictionary for consistent output format
    probabilities_dict = {'probabilities': probabilities}

    # Return both the labels and the probability dictionary
    return labels, probabilities_dict

In [ ]:
# Cluster positive UMAP embeddings using HDBSCAN
positive_labels, _ = cluster_with_hdbscan(positive_umap_embeddings, min_cluster_size=70, min_samples=21)

In [ ]:
# Cluster negative UMAP embeddings using HDBSCAN
negative_labels, _ = cluster_with_hdbscan(negative_umap_embeddings, min_cluster_size=25, min_samples=5)

<h3>6.6 Visualizing Clusters with UMAP and HDBSCAN</h3>

In [ ]:
def plot_umap_clusters(umap_embeddings, labels, category, title='UMAP + HDBSCAN Clusters'):
    """
    Plot UMAP embeddings colored by cluster labels.

    Parameters
    ----------
    umap_embeddings : ndarray of shape (n_samples, 2)
        The 2D UMAP-reduced embeddings.

    labels : array-like of shape (n_samples,)
        Cluster labels assigned to each data point (e.g., from HDBSCAN).
        The label -1 typically represents noise or unclustered points.

    title : str, optional (default='UMAP + HDBSCAN Clusters')
        Title to display on the plot.

    Returns
    -------
    None
        Displays the plot of UMAP embeddings with cluster coloring.
    """
    # Create a figure and axis using the object-oriented interface
    fig, ax = plt.subplots(figsize=(10, 6))

    # Get the unique cluster labels (including noise)
    unique_labels = set(labels)

    # Generate a color palette with as many colors as unique labels
    colors = sns.color_palette('hsv', len(unique_labels))

    # Plot each cluster separately
    for label, color in zip(unique_labels, colors):
        # Create a mask for points belonging to the current cluster
        mask = labels == label
        # Assign a label name: "Cluster X" or "Noise" if label == -1
        label_name = f"Cluster {label}" if label != -1 else "Noise"
        # Scatter plot for the current cluster
        ax.scatter(umap_embeddings[mask, 0], umap_embeddings[mask, 1],
                   c=[color], label=label_name, alpha=0.7, s=40)

    # Set axis labels
    ax.set_xlabel("UMAP Dimension 1", fontsize=12)
    ax.set_ylabel("UMAP Dimension 2", fontsize=12)

    # Set the plot title
    ax.set_title(title, fontfamily='Arial Rounded MT Bold', fontsize=16, pad=15)

    # Configure legend
    ax.legend(title='Clusters', loc='best', frameon=True, fontsize=10, title_fontsize=11)

    # Configure grid
    ax.grid(visible=True, linestyle='--', color='gray', alpha=0.7)

    # Adjust layout and display the plot
    fig.tight_layout()
    plt.show()

    # Save the plot to a standardized output path using the category
    output_path = os.path.join("..", "results", "figures", "topic_modeling", f"{category}_umap_clusters.png")
    save_plot(fig, output_path)

In [ ]:
# Plot UMAP projection of positive review embeddings, colored by their assigned cluster labels
plot_umap_clusters(positive_umap_embeddings, positive_labels, category="positive", title='Clusters of Positive Reviews')

In [ ]:
# Plot UMAP projection of negative review embeddings, colored by their assigned cluster labels
plot_umap_clusters(negative_umap_embeddings, negative_labels, category="negative", title='Clusters of Negative Reviews')

<h3>6.7 Group Reviews by Cluster (Excluding Noise)</h3>

In [ ]:
def export_results_json(data, file_path):
    """
    Export results dictionary to a JSON file, ensuring JSON compatibility.

    This function converts any non-native Python keys (e.g., numpy.int64, numpy.float64)
    and values (e.g., numpy types, arrays) into standard Python types, and then
    writes the result into a JSON file at the specified path.

    Args:
        data (dict): Dictionary containing the results to be exported. Keys can be
            cluster labels or other identifiers, and values can include lists or
            nested structures.
        file_path (str): Full path (including the file name) where the JSON file
            will be saved.

    Returns:
        None

    Raises:
        TypeError: If an object in the dictionary cannot be serialized to JSON.

    Note:
        Requires `numpy` for type checking and conversion.
    """
    
    def convert(o):
        if isinstance(o, (np.integer, np.int64)):
            return int(o)
        elif isinstance(o, (np.floating, np.float32, np.float64)):
            return float(o)
        elif isinstance(o, np.ndarray):
            return o.tolist()
        else:
            raise TypeError(f"Object of type {type(o).__name__} is not JSON serializable")

    # Convert dictionary keys to native Python types if needed
    converted_data = {
        int(key) if isinstance(key, (np.integer, np.floating)) else key: value
        for key, value in data.items()
    }

    # Write the data to JSON file with proper formatting
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(converted_data, f, ensure_ascii=False, indent=2, default=convert)

In [ ]:
def group_texts_by_cluster_no_noise(texts, labels, exclude_noise=True):
    """
    Groups review texts by their corresponding cluster labels.
    
    When `exclude_noise=True`, all texts labeled as noise (-1) are ignored, so the result
    includes only clean, clustered data.

    Args:
        texts (Iterable[str]): List of review texts.
        labels (Iterable[int]): Cluster labels assigned to each text (e.g., from HDBSCAN).
        exclude_noise (bool, optional): If True, excludes texts labeled as noise (-1). Defaults to True.

    Returns:
        dict: Dictionary where keys are cluster labels and values are lists of texts belonging to each cluster.

    Example:
        >>> grouped = group_texts_by_cluster(my_reviews, my_labels)
        >>> grouped[0]  # All texts from cluster 0
    """
    clustered_texts = defaultdict(list)

    for text, label in zip(texts, labels):
        # If exclude_noise is True, skip texts labeled as noise (-1)
        if exclude_noise and label == -1:
            continue
        # Check if the text is a valid, non-empty string
        if isinstance(text, str) and text.strip():
            # Remove any leading/trailing whitespace and add the text to the corresponding cluster
            clustered_texts[label].append(text.strip())

    return dict(clustered_texts)

In [ ]:
# Group positive reviews by cluster while excluding noise points
positive_clustered_reviews = group_texts_by_cluster_no_noise(positive_reviews, positive_labels)

In [ ]:
# Group negative reviews by cluster while excluding noise points
negative_clustered_reviews = group_texts_by_cluster_no_noise(negative_reviews, negative_labels)

In [ ]:
# Export the positive clustered reviews data to a JSON file in the specified path.
export_results_json(
    positive_clustered_reviews, 
    os.path.join("..", "results", "clustering", "positive_clustered_reviews.json")
)

In [ ]:
# Export the negative clustered reviews data to a JSON file in the specified path.
export_results_json(
    negative_clustered_reviews, 
    os.path.join("..", "results", "clustering", "negative_clustered_reviews.json")
)

<h2>8. Topic Extraction and Clustering Using BERTimbau</h2>

In [ ]:
def get_embedding_batch(texts, batch_size=32):
    """
    Computes BERT embeddings for a batch of texts.

    Args:
        texts (list of str): List of input texts.
        batch_size (int, optional): Number of texts to process at a time. Default is 32.

    Returns:
        list of torch.Tensor: List of sentence embeddings.
    """
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]

        # Tokenize the batch with padding and truncation
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        # Extract hidden states from the last layer
        embeddings = outputs.last_hidden_state  # (batch_size, seq_len, hidden_dim)

        # Expand attention mask to match the embeddings shape
        attention_mask = inputs['attention_mask'].unsqueeze(-1).expand(embeddings.size())

        # Apply attention mask to embeddings
        masked_embeddings = embeddings * attention_mask

        # Sum embeddings across the sequence length dimension
        summed = masked_embeddings.sum(dim=1)

        # Count the number of valid (non-padded) tokens
        counts = attention_mask.sum(dim=1)

        # Compute the mean embedding per sentence
        mean_embeddings = (summed / counts).cpu()

        # Store the computed embeddings
        all_embeddings.extend(mean_embeddings)

    return all_embeddings

In [ ]:
def generate_ngrams(text, ngram_range=(1, 3)):
    """
    Generates n-grams from a given text.

    Args:
        text (str): Input text.
        ngram_range (tuple, optional): Range of n-grams to generate (min_n, max_n). Default is (1, 3).

    Returns:
        list of str: Unique n-grams extracted from the text.
    """
    words = text.split()
    ngrams = []

    # Generate n-grams for each value of n in the given range
    for n in range(ngram_range[0], ngram_range[1] + 1):
        for i in range(len(words) - n + 1):
            ngram = ' '.join(words[i:i + n])
            ngrams.append(ngram)

    return list(set(ngrams))  # Remove duplicates

In [ ]:
def truncate_text(text, max_tokens=512):
    """
    Truncates a text to a maximum number of tokens.

    Args:
        text (str): Input text.
        max_tokens (int, optional): Maximum number of tokens allowed. Default is 512.

    Returns:
        str: Truncated text converted back to a string.
    """
    tokens = tokenizer.tokenize(text)

    # Return the original text if it is within the token limit
    if len(tokens) <= max_tokens:
        return text

    # Truncate tokens to the maximum limit
    truncated_tokens = tokens[:max_tokens]
    return tokenizer.convert_tokens_to_string(truncated_tokens)

In [ ]:
def extract_keywords_bertimbau(text, top_n=5, ngram_range=(1, 3)):
    """
    Extracts the most relevant keywords from a given text using BERTimbau embeddings.

    Args:
        text (str): Input text.
        top_n (int, optional): Number of top keywords to return. Default is 5.
        ngram_range (tuple, optional): Range of n-grams to consider. Default is (1, 3).

    Returns:
        list: A list of tuples containing the top keywords and their similarity scores.
    """
    # Generate the document embedding
    doc_embedding = get_embedding_batch([text])[0]  # get_embedding_batch returns a list of embeddings

    # Generate candidate n-grams from the text
    candidates = generate_ngrams(text, ngram_range=ngram_range)

    # Compute embeddings for the candidate n-grams
    candidate_embeddings = get_embedding_batch(candidates)  # Returns embeddings for all candidates

    # Compute cosine similarity between the document embedding and candidate embeddings
    similarities = cosine_similarity([doc_embedding], candidate_embeddings).flatten()

    # Rank candidates by similarity score in descending order
    ranked = sorted(zip(candidates, similarities), key=lambda x: x[1], reverse=True)

    return ranked[:top_n]

In [ ]:
def extract_keywords_from_clusters_bertimbau(clustered_texts, top_n=5, ngram_range=(2, 3)):
    """
    Extracts the most relevant keywords for each text cluster using BERTimbau embeddings.

    Args:
        clustered_texts (dict): Dictionary where keys are cluster IDs and values are lists of texts.
        top_n (int, optional): Number of top keywords to extract for each cluster. Default is 5.
        ngram_range (tuple, optional): Range of n-grams to consider. Default is (2, 3).

    Returns:
        dict: A dictionary mapping each cluster ID to a list of extracted keywords and their scores.
    """
    cluster_keywords = {}

    # Iterate over each cluster
    for cluster_id, texts in tqdm(clustered_texts.items(), desc="Extracting cluster keywords"):
        combined_text = ' '.join(texts)  # Combine all texts within the cluster
        keywords = extract_keywords_bertimbau(combined_text, top_n=top_n, ngram_range=ngram_range)
        cluster_keywords[cluster_id] = keywords  # Store keywords for each cluster

    return cluster_keywords

In [ ]:
def convert_dict_to_dataframe(topic_dict):
    """
    Converts a dictionary of topic clusters into a structured pandas DataFrame.

    Args:
        topic_dict (dict): A dictionary where keys are cluster labels (integers) and 
                           values are lists of tuples containing topic phrases and their scores.

    Returns:
        pd.DataFrame: A DataFrame with columns 'Cluster', 'Topic', and 'Score'.
    """
    data = []
    
    for cluster, topics in topic_dict.items():
        for topic, score in topics:
            data.append({'Cluster': cluster, 'Topic': topic, 'Score': score})
    
    return pd.DataFrame(data)

<h3>8.1 Keyword Extraction and Clustering for Positive Reviews</h3>

In [ ]:
# Extracts top 3 keywords for each cluster in the positive reviews
positive_cluster_topics = extract_keywords_from_clusters_bertimbau(  
    clustered_texts=positive_clustered_reviews,  
    top_n=5,  
    ngram_range=(2, 3)  
)

In [ ]:
# Export the extracted topics from positive clusters to a JSON file at the specified path.
export_results_json(
    positive_cluster_topics, 
    os.path.join("..", "results", "clustering", "positive_cluster_topics.json")
)

In [ ]:
# Converts the positive cluster topics into a DataFrame
positive_topics_df = convert_dict_to_dataframe(positive_cluster_topics)

In [ ]:
positive_topics_df.sort_values(by='Cluster')

<h3>8.2 Keyword Extraction and Clustering for Negative Reviews</h3>

In [ ]:
# Extracts top 3 keywords for each cluster in the negative reviews
negative_cluster_topics = extract_keywords_from_clusters_bertimbau(  
    clustered_texts=negative_clustered_reviews,  
    top_n=5,  
    ngram_range=(2, 3)  
)

In [ ]:
# Export the extracted topics from negative clusters to a JSON file at the specified path.
export_results_json(
    negative_cluster_topics, 
    os.path.join("..", "results", "clustering", "negative_cluster_topics.json")
)

In [ ]:
# Converts the negative cluster topics into a DataFrame
negative_topics_df = convert_dict_to_dataframe(negative_cluster_topics)

In [ ]:
negative_topics_df.sort_values(by='Cluster')

<h2>9. Summarize Topic Clusters</h2>

In [ ]:
def generate_cluster_labels(cluster_dict, top_n_keywords=3):
    """
    Generates label suggestions for each cluster based on its most representative phrases
    using KeyBERT to extract keywords.

    Args:
        cluster_dict (dict): Dictionary formatted as {cluster_id: [(phrase, score), ...]}.
            Each value is a list of tuples containing a representative phrase and its importance score.
        top_n_keywords (int): Number of keywords to extract and use in the suggested label.

    Returns:
        dict: Dictionary with cluster_id as keys and suggested labels as values.
    """
    # Initialize the KeyBERT model using a lightweight sentence transformer
    kw_model = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')
    cluster_labels = {}

    # Iterate through each cluster and generate a label
    for cluster_id, phrases_scores in cluster_dict.items():
        # Select the top-N most representative phrases for the current cluster
        top_phrases = [phrase for phrase, _ in phrases_scores[:5]]

        # Join the phrases into a single text block
        joined_text = " ".join(top_phrases)

        # Extract keywords from the combined text
        keywords = kw_model.extract_keywords(
            joined_text,
            keyphrase_ngram_range=(2, 3),
            top_n=top_n_keywords
        )

        # Combine the keywords into a comma-separated label string
        label = ", ".join([kw for kw, _ in keywords])

        # Assign the label to the current cluster
        cluster_labels[cluster_id] = label

    return cluster_labels

In [ ]:
def create_topic_summary_df(keybert_topics: dict, manual_summaries: dict) -> pd.DataFrame:
    """
    Creates a DataFrame combining KeyBERT topics (already processed) and manual summaries by cluster,
    and sorts the result by Cluster ID.

    Args:
        keybert_topics (dict): A dictionary where each key is a cluster ID and 
                               each value is a string or list of keywords already processed.
        manual_summaries (dict): A dictionary with manual summaries for each cluster ID.

    Returns:
        pd.DataFrame: DataFrame with 'Cluster ID', 'KeyBERT Topics', and 'Manual Summary',
                      sorted by Cluster ID.
    """
    data = []
    for cluster_id, keywords in keybert_topics.items():
        if isinstance(keywords, list):
            top_keywords = ", ".join(keywords)
        else:
            top_keywords = keywords  # assume it's already a string

        summary = manual_summaries.get(cluster_id, "No summary")
        data.append({
            "Cluster ID": cluster_id,
            "KeyBERT Topics": top_keywords,
            "Manual Summary": summary
        })

    return pd.DataFrame(data).sort_values(by="Cluster ID").reset_index(drop=True)

<h3>9.1 Topic Summarization of Positive Clusters</h3>

In [ ]:
# Generate representative labels for each positive cluster using the top keywords
positive_cluster_labels = generate_cluster_labels(positive_cluster_topics)

In [ ]:
# Print each cluster ID and its corresponding label in sorted order
for cluster_id in sorted(positive_cluster_labels):
    print(f"Cluster {cluster_id}: {positive_cluster_labels[cluster_id]}")

In [ ]:
# Manually written summaries for each positive cluster
positive_cluster_summaries = {
    0: "Excellent veterinarians with great care and attention.",
    1: "Caring and friendly service; highly recommended.",
    2: "Wonderful place with amazing staff and products.",
    3: "Loved by pet owners; affectionate and reliable service.",
    4: "Attentive professionals with a focus on detail and care.",
    5: "High-quality service and good variety of pet food.",
    6: "Very affectionate grooming and positive customer experience.",
    7: "Outstanding service with warmth and appreciation.",
    8: "Impeccable and affectionate work; strongly recommended.",
    9: "Pets love the place; trusted and beloved service.",
    10: "Impressive service with knowledgeable and kind staff."
}

In [ ]:
# Create a DataFrame combining KeyBERT topics and manual summaries for positive clusters
positive_topics_df = create_topic_summary_df(
    keybert_topics=positive_cluster_labels,       # Dictionary with KeyBERT topics per cluster
    manual_summaries=positive_cluster_summaries   # Dictionary with manual summaries per cluster
)

<h3>9.2 Topic Summarization of Negative Clusters</h3>

In [ ]:
# Generate representative labels for each negative cluster using the top keywords
negative_cluster_labels = generate_cluster_labels(negative_cluster_topics)

In [ ]:
# Print each cluster ID and its corresponding label in sorted order
for cluster_id in sorted(negative_cluster_labels):
    print(f"Cluster {cluster_id}: {negative_cluster_labels[cluster_id]}")

In [ ]:
# Manually written summaries for each negative cluster
negative_cluster_summaries = {
    0: "Poor service and customer dissatisfaction.",
    1: "Negative experiences related to pet mistreatment.",
    2: "Health issues and concerns with pet treatment.",
    3: "Disappointment with care and service provided.",
    4: "Sarcastic remarks and disbelief about diagnosis."
}

In [ ]:
# Create a DataFrame combining KeyBERT topics and manual summaries for negative clusters
negative_topics_df = create_topic_summary_df(
    keybert_topics=negative_cluster_labels,       # Dictionary with KeyBERT topics per cluster
    manual_summaries=negative_cluster_summaries   # Dictionary with manual summaries per cluster
)

<h3>9.3 Save the Results to a CSV File</h3>

In [ ]:
# Define the path for saving the CSV file
POSITIVE_TOPICS_PATH = os.path.join("..", "results", "clustering", "positive_cluster_topic_summaries.csv")
NEGATIVE_TOPICS_PATH = os.path.join("..", "results", "clustering", "negative_cluster_topic_summaries.csv")

In [ ]:
# Save the dataframe to a CSV file
positive_topics_df.to_csv(POSITIVE_TOPICS_PATH, index=False, sep=";", encoding="utf-8")
negative_topics_df.to_csv(NEGATIVE_TOPICS_PATH, index=False, sep=";", encoding="utf-8")

<h2>10. Conclusion</h2>

This topic modeling approach enabled the identification of recurring themes in customer reviews related to pet services. The pipeline followed these main steps:

- **BERTimbau embeddings** were used to generate semantic representations of each review in Brazilian Portuguese.  
- **HDBSCAN** was applied to cluster the embeddings based on semantic density, without needing to predefine the number of topics.  
- **BERTimbau** was used again to extract relevant keywords from each cluster.  
- **KeyBERT** summarized and refined the keywords, producing interpretable labels for each topic.

Negative reviews frequently focused on issues such as poor service, health concerns, and dissatisfaction with care. Positive reviews, on the other hand, highlighted attentive professionals, trust in pet handling, and overall satisfaction. These patterns were captured in concise summaries for each cluster, allowing for a clear distinction between commonly praised and criticized aspects of pet service businesses.

The use of **HDBSCAN** was particularly effective in this context. One of the reasons for choosing HDBSCAN was the uncertainty about how many distinct topics would be present in the reviews. Unlike traditional clustering methods, HDBSCAN does not require the number of clusters to be specified in advance. Instead, it automatically detects the number of clusters based on the semantic density of the data, making it a more flexible and appropriate choice for this type of unstructured, complex text. 

To ensure well-formed clusters, all HDBSCAN and UMAP parameters were fine-tuned using **Optuna**, a hyperparameter optimization framework. During the parameter tuning process, the initial trials were performed using broad, random search intervals. This was necessary to explore the overall landscape of possible configurations and understand how different parameter ranges influenced the clustering quality. Once the initial search revealed promising regions in the parameter space, the search intervals were refined to focus on those narrower, more optimal ranges. This two-step tuning strategy helped improve the efficiency of the optimization process and resulted in more coherent and well-separated topic clusters, as measured by the **Silhouette Score**.

While higher similarity scores often indicate that a topic phrase is semantically close to the cluster content, they don't necessarily guarantee that the topic is informative or diverse. On the **topic extraction using BERTimbau** step, this becomes particularly relevant: especially when using larger n-grams (e.g., bigrams or trigrams), scores tend to increase naturally due to richer context. However, this can lead to **redundant** or **overly generic phrases** that fail to summarize the core ideas effectively.

It is important to acknowledge a key limitation of this project: due to restrictions imposed by the API, only a limited number of reviews could be retrieved per business. This constraint reduced the diversity and volume of textual data available for analysis. As a result, the topics extracted in this study may not fully capture the broader range of customer experiences and opinions within the pet services sector.

<h2>11. Next Steps</h2>

Building upon the current results, several directions could enhance the robustness and relevance of the topic modeling process:

- **Expand the dataset**  
  Collect a broader and more diverse set of reviews, ideally overcoming current API limitations. This would improve topic coverage and provide a more representative view of customer experiences in the pet services sector.

- **Apply the methodology to new domains or regions**  
  Extend the approach to different geographical areas or business types within the pet care industry to compare topic distributions across contexts.

- **Improve topic interpretability**  
  Integrate filtering and ranking strategies that balance **semantic cohesion** and **diversity**, helping to surface more informative and actionable clusters.

- **Refine cluster quality**  
  Use alternative validation metrics and qualitative assessments to ensure that clusters remain meaningful, distinct, and robust.

- **Enhance hyperparameter optimization**  
  Adopt more advanced optimization strategies. In this project, a two-step approach was used: broad random exploration followed by narrowing the search space. Future iterations could include:
  
  - **Bayesian optimization with early pruning**  
    Use Optuna's pruning mechanism to halt underperforming trials early based on intermediate metrics (e.g., partial Silhouette Scores), improving computational efficiency.

  - **Multi-objective optimization**  
    Optimize for multiple objectives simultaneously (e.g., cohesion and diversity) to produce more interpretable and well-balanced clusters.

  - **Additional evaluation metrics**  
    Incorporate metrics beyond the Silhouette Score, such as the **Davies–Bouldin index** or clustering stability scores, to gain a more comprehensive understanding of cluster quality.

- **Explore automatic topic labeling**  
  Investigate alternatives or complements to KeyBERT to reduce the reliance on manual interpretation, especially in large-scale applications.

These improvements would not only increase the reliability of topic modeling results but also open the door for deeper and more scalable insights across customer-generated content.